In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:90% !important; }</style>")

In [2]:
import os
import numpy as np
  
import lsst.afw.display.ds9 as ds9
import lsst.afw.geom as afwGeom
import lsst.afw.detection as afwDetect
import lsst.afw.geom.ellipses as afwEllipse
import lsst.afw.cameraGeom as cameraGeom
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.afw.display.ds9 as display
import lsst.afw.cameraGeom.utils as camGeomUtils

from functions import *

In [3]:
CURRENT_FRAME = 0
def GetNextFrame():
    global CURRENT_FRAME
    CURRENT_FRAME += 1
    return CURRENT_FRAME-1

In [4]:
info, dummy = ParseLogfile('/home/mfisherlevine/first_processing_j1.log')

In [5]:
def PrintSuccesses(info):
    for id in info.keys():
        if info[id]['PASSED'] == True:
            print id
#         print info[id]['object'] + '\n' + info[id]['OUTCOME'] +'\n' +  info[id]['LINE_ABOVE_FATAL'] + '\n'
        
def PrintFailures(info):
    for id in info.keys():
        if info[id]['PASSED'] == False:
            print
#         print info[id]['object'] + '\n' + info[id]['OUTCOME'] +'\n' +  info[id]['LINE_ABOVE_FATAL'] + '\n'

def ds9_one_visit(rerun, visit, product, ccdname, binning=8, date='2015-11-25',filter='r', showMaskPlanes=False):
    import lsst.daf.persistence as dafPersist
    import lsst.afw.display.ds9 as ds9
#     import lsst.afw.cameraGeom.utils as camGeomUtils

    butler = dafPersist.Butler('/nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/processing/rerun/first_processing_j1/')
#     camera = butler.get("camera")

    exp = butler.get(product, visit=visit, immediate=True)#date=date, filter=filter, ccdnum=getCCDNumFromName(ccdname),
    
    if showMaskPlanes:
        image = exp
    else:
        image = exp.getMaskedImage().getImage()
    
    title = 'rerun %s, visit %s, %s'%(rerun, visit, product)
    ds9.mtv(image, frame=GetNextFrame(), title=title)


In [ ]:
PrintSuccesses(info)

In [6]:
butler = dafPersist.Butler('/nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/processing/rerun/first_processing_j1/')

# camera = butler.get("camera")

date = '2016-05-04'
visit = 1100
product = 'calexp'
# product = 'postISRCCD'
# product = 'raw'
# filter = 'r'


im = butler.get(product, visit=visit)#, ccdnum=ccdnum)

# ds9.mtv(im.getMaskedImage().getImage(), frame=GetNextFrame(), title='%s,%s'%(visit,product))
ds9.mtv(im, frame=GetNextFrame(), title='%s,%s'%(visit,product))


In [ ]:
import sys
try:
    import lsstDebug

    print "Importing debug settings..."
    def DebugInfo(name):
        di = lsstDebug.getInfo(name)
        if name in (
            "lsst.meas.astrom.astrometry",
            "lsst.meas.astrom.matchOptimisticB",
            "lsst.meas.astrom.anetAstrometry",
            "lsst.meas.astrom.anetBasicAstrometry",
            ):
            di.display = 3
            di.showLinear = True
            di.verbose = True
            di.frame = 1
            di.frame1 = 1
            di.frame2 = 2
            di.frame3 = 3
            di.pause = True
            di.scatterPlot = 2
            di.displaySources = True
        return di

    lsstDebug.Info = DebugInfo

except ImportError:
    print >> sys.stderr, "Unable to import lsstDebug; not setting display intelligently"

In [ ]:
dataId = dict(visit=496547, ccdnum=28)

src = butler.get("icSrc", dataId, immediate=True)
calexp = butler.get("calexp", dataId)
if True:
    ds9.mtv(calexp.getMaskedImage().getImage(), frame=62)
else:
    ds9.erase()
with ds9.Buffering():
    i = 1
    for s in src:
        if s.getFootprint().getNpix() > 1:
            print i
            i += 1
            ellip = s.getShape()
            ellip = afwEllipse.Axes(ellip)

            if ellip.getDeterminantRadius() > 10:
                ds9.dot('o', *s.getCentroid(), size=10, frame=62)
                print s.getCentroid()